## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.


In [45]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
import os
import imp
import Advanced_Lane_Lines as all

%matplotlib qt

In [92]:
imp.reload(Advanced_Lane_Lines)

ret, mtx, dist, rvecs, tvecs = all.calibrate_camera()

all.set_calibrate_camera_result(ret, mtx, dist, rvecs, tvecs)

print(ret)
print(mtx)
print(dist)
# print(rvecs)
# print(tvecs)

s1 = [180,719]
s2 = [555, 470]
s3 = [725, 470]
s4 = [1130,719]

d1 = [280,719]
d2 = [280,  0]
d3 = [1000,  0]
d4 = [1000, 719]

src = np.array([s1,s2,s3,s4],dtype = "float32")
dst = np.array([d1,d2,d3,d4],dtype = "float32")

M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)

1.0298153371058987
[[1.15777930e+03 0.00000000e+00 6.67111054e+02]
 [0.00000000e+00 1.15282291e+03 3.86128938e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
[[-0.24688775 -0.02373133 -0.00109842  0.00035108 -0.00258571]]


In [78]:
# def my_cal_perspective_transform_mtx():
#     M = cv2.getPerspectiveTransform(src, dst)
#     Minv = cv2.getPerspectiveTransform(dst, src)
    
#     return M, Minv

In [27]:
cwd = os.getcwd()
img = cv2.imread(cwd + '/camera_cal/calibration3.jpg')

# cv2.imshow('calibration3.jpg',img)

img_undistort = all.undistort_img(img, mtx, dist)

# cv2.imshow('img_undistort', img_undistort)

perspective_transform_mtx = all.cal_perspective_transform_mtx(img, 9, 6, mtx, dist)

print(perspective_transform_mtx)

print(img.shape)

warped = cv2.warpPerspective(img_undistort, perspective_transform_mtx, img.shape[1::-1], cv2.INTER_LINEAR)

cv2.imshow('warped', warped)


[[ 1.23176607e+00  2.89559832e-01 -1.55916586e+02]
 [-2.60900799e-02  1.31677308e+00  2.97882835e+01]
 [-6.38198199e-05  4.89794342e-04  1.00000000e+00]]
(720, 1280, 3)


In [84]:
cwd = os.getcwd()

# img = mpimg.imread(cwd + '/test_images/straight_lines2.jpg')
img = mpimg.imread(cwd + '/test_images/test6.jpg')

# plt.imshow(img)

img_undistort = all.undistort_img(img, mtx, dist)

# plt.imshow(img)

plt.imshow(img_undistort)

plt.show()


In [85]:
img_result = all.my_pipline(img, mtx, dist)
print(img_result.shape)
plt.imshow(img_result)

plt.show()


720
640.0
336.65532491640727
1028.2217252529924
682.4385250846999
-0.22431791830484235
2118.264373543409
14178.966936342767
-0.22431791830484235
(720, 1280, 3)


In [67]:
img_bin_sobel =  all.abs_sobel_thresh(img_undistort, orient='x', thresh_min=10, thresh_max=200)

print(img_bin_sobel)

# plt.imshow(img_bin_sobel, cmap='gray')

img_bin_mag =  all.mag_thresh(img_undistort, sobel_kernel=3, mag_thresh=(20, 200))

# plt.imshow(img_bin_mag, cmap='gray')

img_bin_dir =  all.dir_threshold(img_undistort, sobel_kernel=3, thresh=(0.7, 1.3))

# plt.imshow(img_bin_dir, cmap='gray')

img_bin_hls =  all.hls_select(img_undistort, thresh=(25, 200))

# plt.imshow(img_bin_hls, cmap='gray')

combined = np.zeros_like(img_bin_sobel)
# combined[(((img_bin_sobel == 1) | (img_bin_mag == 1)) & (img_bin_hls == 1)) & (img_bin_dir == 1)] = 1
combined[((img_bin_sobel == 1) & (img_bin_mag == 1)) & ((img_bin_dir == 1) & (img_bin_hls == 1))] = 1
# combined[((img_bin_sobel == 1) )] = 1

# plt.imshow(combined, cmap='gray')

combined_bin, color_bin = all.make_binary(img_undistort)

# plt.imshow(combined_bin, cmap='gray')
# plt.imshow(color_bin, cmap='gray')


# (120,539),(480, 310), (485, 310), (900,539)
# img_cpy = np.copy(img_undistort)
img_cpy = np.copy(img_undistort)

# plt.imshow(img_cpy)

print(img_cpy.shape)

s1 = [180,719]
s2 = [555, 470]
s3 = [725, 470]
s4 = [1130,719]

d1 = [280,719]
d2 = [280,  0]
d3 = [1000,  0]
d4 = [1000, 719]

w = 1
# cv2.line(img_cpy, (180,719), (550, 470), (255,0,0),w) 
# cv2.line(img_cpy, (550, 470), (740, 470), (255,0,0),w) 
# cv2.line(img_cpy, (740, 470), (1130,719), (255,0,0),w) 

cv2.line(img_cpy, s1, s2, (255,0,0),w) 
cv2.line(img_cpy, s2, s3, (255,0,0),w) 
cv2.line(img_cpy, s3, s4, (255,0,0),w) 

# plt.imshow(img_cpy)

# src = np.array([[180,719],[550,470],[740, 470],[1130,719]],dtype = "float32")
# dst = np.array([[280,719],[280,  0],[1000,  0],[1000, 719]],dtype = "float32")

# src = np.array([s1,s2,s3,s4],dtype = "float32")
# dst = np.array([d1,d2,d3,d4],dtype = "float32")

# M = cv2.getPerspectiveTransform(src, dst)
# Minv = cv2.getPerspectiveTransform(dst, src)
    
img_warper = all.warper(img_cpy, src, dst)

# plt.imshow(img_warper)

# img_warper = cv2.warpPerspective(img_cpy, perspective_transform_mtx, img_cpy.shape[1::-1], cv2.INTER_LINEAR)

# plt.imshow(img_warper)

colored_comb_bin = np.dstack((combined, combined, combined)) * 255

# cv2.line(colored_comb_bin, s1, s2, (255,0,0),w) 
# cv2.line(colored_comb_bin, s2, s3, (255,0,0),w) 
# cv2.line(colored_comb_bin, s3, s4, (255,0,0),w) 

# plt.imshow(colored_comb_bin, cmap='gray')

img_warper = all.warper(combined, src, dst)

# plt.imshow(img_warper, cmap='gray')

colored_warped = np.dstack((img_warper, img_warper, img_warper)) * 255

# plt.imshow(colored_warped, cmap='gray')

histogram = all.hist(colored_warped)

# plt.plot(histogram)

# leftx, lefty, rightx, righty, out_img = find_lane_pixels(img_warper)
leftx, lefty, rightx, righty, out_img = all.find_lane_pixels(img_warper* 255) 

print(leftx)
print(lefty)
print(rightx)
print(righty)

# plt.imshow(out_img)

img_fit = all.fit_polynomial(img_warper* 255)

# plt.imshow(img_fit)

left_fit, right_fit, ploty = all.fit_poly(img_warper.shape, leftx, lefty, rightx, righty)

result = all.search_around_poly(img_warper)

# plt.imshow(result)

# left_curverad, right_curverad = measure_curvature_pixels()

left_curverad, right_curverad = all.my_measure_curvature_pixels(img_warper)

print(left_curverad)
print(right_curverad)

left_curverad, right_curverad = all.my_measure_curvature_real(img_warper)

print(left_curverad)
print(right_curverad)

# plt.imshow(img)
# plt.imshow(img_undistort)
# plt.imshow(img_warper)

# img_final = my_draw(img, img_undistort, img_warper, Minv,left_fitx,right_fitx,ploty)
img_final = all.my_draw(img, img_undistort, img_warper, Minv,left_fit,right_fit,ploty)

# img = cv2.imread('caijian.jpg')
font = cv2.FONT_HERSHEY_SIMPLEX

imgzi = cv2.putText(img_final, 'Curve Radius: left: %fm, right =%fm' %(left_curverad,right_curverad) , (50, 50), font, 1.2, (255, 255, 255), 2)

plt.imshow(img_final)

plt.show()

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(720, 1280, 3)
[354 355 356 ... 341 342 343]
[640 640 640 ...  79  79  79]
[ 914  912  959 ... 1019 1020 1021]
[706 707 707 ... 319 319 319]
3421.986977479236
1342.6051933050703
1121.1698856552575
421.4621984789208


In [86]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [93]:
white_output = cwd + '/output_video/P2.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip(cwd + "/project_video.mp4")
white_clip = clip1.fl_image(all.my_pipline) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

720
640.0
358.03782837194467
1052.148761318555
705.0932948452498
-0.34406455846774914
1290.0378256752128
1562.5774852668294
-0.34406455846774914
Moviepy - Building video D:\hzf\udacity\project\CarND-Advanced-Lane-Lines/output_video/P2.mp4.
Moviepy - Writing video D:\hzf\udacity\project\CarND-Advanced-Lane-Lines/output_video/P2.mp4



t:   0%|                                                                    | 2/1260 [00:00<03:41,  5.68it/s, now=None]

720
640.0
358.03782837194467
1052.148761318555
705.0932948452498
-0.34406455846774914
1290.0378256752128
1562.5774852668294
-0.34406455846774914


t:   0%|▏                                                                   | 3/1260 [00:00<05:55,  3.54it/s, now=None]

720
640.0
358.16572326313116
1056.2474600821922
707.2065916726617
-0.3552348416983548
1159.648130898259
3263.0032024069737
-0.3552348416983548


...

720
640.0
378.15019234261916
1073.1774345138463
725.6638134282327
-0.45279444240637307
7609.205426469914
5490.802222049659
-0.45279444240637307


t: 100%|█████████████████████████████████████████████████████████████████| 1260/1260 [12:30<00:00,  1.46it/s, now=None]

720
640.0
374.57222997482546
1077.499062885598
726.0356464302117
-0.45475984541683345
268059.5576498204
2224.664695704661
-0.45475984541683345


720
640.0
373.7182625118166
1074.341474804877
724.0298686583468
-0.4441578771941188
8652.791973874506
2991.9106052859233
-0.4441578771941188


Moviepy - Done !
Moviepy - video ready D:\hzf\udacity\project\CarND-Advanced-Lane-Lines/output_video/P2.mp4
Wall time: 12min 33s


In [88]:
print(white_output)
# HTML("""
# <video width="1280" height="720" controls>
#   <source src="{0}">
# </video>
# """.format(white_output))

HTML("""
<video width="1280" height="720" controls>
  <source src="D:\\hzf\\udacity\\project\\CarND-Advanced-Lane-Lines\\output_video\\P2.mp4">
</video>
""")

D:\hzf\udacity\project\CarND-Advanced-Lane-Lines/output_video/P2.mp4


In [20]:
img = pipeline(img, s_thresh=(170, 255), sx_thresh=(20, 100))

print(img)
plt.imshow(img, cmap='gray')
# plt.imshow(img)

plt.show()

[[0 1 0 ... 1 1 1]
 [0 1 0 ... 1 1 1]
 [0 1 0 ... 1 1 0]
 ...
 [0 1 0 ... 1 1 0]
 [0 1 1 ... 1 1 0]
 [0 1 1 ... 1 1 0]]


In [68]:

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
# print(objp)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
# print(objp)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

cwd = os.getcwd()
print(cwd)
img = cv2.imread(cwd + '/camera_cal/calibration3.jpg')
# print(img)
# cv2.imshow('calibration3',img)

gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

print(gray.shape)

# cv2.imshow('calibration3',gray)

# Find the chessboard corners
ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

# If found, add object points, image points
if ret == True:
#     print(objp)
#     print(corners)
    objpoints.append(objp)
    imgpoints.append(corners)

    # Draw and display the corners
#     img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
#     cv2.imshow('img',img)
#             cv2.waitKey(500)

#     image = mpimg.imread('camera_cal/calibration1.jpg')
print(img.shape)
print(img.shape[1::-1])
# return cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

img_undistort = undistort_img(img, mtx, dist)

cv2.imshow('img_undistort', img_undistort)


# cwd = os.getcwd()
# # Make a list of calibration im1ages
# images = glob.glob(cwd + './camera_cal/calibration*.jpg')



# # Make a list of calibration images
# images = glob.glob(cwd + '/camera_cal/calibration*.jpg')

# for index, itrImg in enumerate(images):
#     if (index == 1) :
#         print(itrImg)
# #         img = cv2.imread(itrImg)
#         img = cv2.imread(cwd + '/camera_cal/calibration3.jpg')

#         plt.imshow(img)
# #         cv2.imshow('img',img)
#         undist = cv2.undistort(img, mtx, dist, None, mtx)
#         plt.imshow(undist)




D:\hzf\udacity\project\CarND-Advanced-Lane-Lines
(720, 1280)
(720, 1280, 3)
(1280, 720)
